In [ ]:

import random
import time
import resource

class MovingMagicSquareGA:
    def __init__(self, population_size, mutation_rate, crossover_rate, max_generations):
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.max_generations = max_generations


    def generate_initial_population(self):
        population = []
        while len(population) < self.population_size:
           solution = list(range(1, 10))
           random.shuffle(solution)
           population.append(solution)
        return population

    # Calculate the fitness of a solution.
    def fitness_function(self, solution):
        # Calculate the sums of rows, columns, and diagonals
        row_sums = [sum(solution[i:i+3]) for i in range(0, 9, 3)]
        col_sums = [sum(solution[i::3]) for i in range(3)]
        diagonal_sums = [sum(solution[::4]), sum(solution[2:8:2])]
        target_sum = 15
        fitness = sum(1 for s in row_sums + col_sums + diagonal_sums if s == target_sum)
        return fitness

    # Calculate fitness as the count of sums equal to the target sum
    def select_parents(self, population):
        tournament_size = 2
        parents = []
        while len(parents) < self.population_size:
            tournament = random.sample(population, tournament_size)
            winner = max(tournament, key=self.fitness_function)
            parents.append(winner)
        return parents

    # Perform crossover between two parents.
    def crossover(self, parent1, parent2):
        # Select a random crossover point
        if random.random() < self.crossover_rate:
            crossover_point = random.randint(1, 7)
            child1 = parent1[:crossover_point] + [x for x in parent2 if x not in parent1[:crossover_point]]
            child2 = parent2[:crossover_point] + [x for x in parent1 if x not in parent2[:crossover_point]]
            return child1, child2
        else:
            return parent1, parent2
    # Mutate a solution.
    def mutate(self, solution):
        if random.random() < self.mutation_rate:
            idx_9 = solution.index(9)   # Find the index of 9 in the solution
            possible_swaps = [i for i in range(9) if abs(i - idx_9) == 1 or abs(i - idx_9) == 3]
            swap_idx = random.choice(possible_swaps)
            solution[idx_9], solution[swap_idx] = solution[swap_idx], solution[idx_9]  # Perform the swap

    # Generate offspring using crossover and mutation
    def generate_offspring(self, parents):
       offspring = []
       i = 0
       while i < self.population_size:
            parent1, parent2 = parents[i], parents[i+1]
            child1, child2 = self.crossover(parent1, parent2)
            self.mutate(child1)
            self.mutate(child2)
            offspring.extend([child1, child2])  # Add children to offspring
            i += 2
       return offspring

    # Replace the population with the new offspring.
    def replace_population(self, population, offspring):

        combined_population = population + offspring  # Combine population and offspring
        combined_population.sort(key=self.fitness_function, reverse=True) # Sort by fitness
        new_population = combined_population[:self.population_size]
        return new_population

    # Solve the moving magic square problem using a genetic algorithm.
    def genetic_algorithm(self):

        population = self.generate_initial_population()  # Generate initial population
        generation = 0
        while generation < self.max_generations:
            parents = self.select_parents(population)
            offspring = self.generate_offspring(parents)
            population = self.replace_population(population, offspring)
            best_solution = max(population, key=self.fitness_function)
            if self.fitness_function(best_solution) >= 0:
                row_sums = [sum(best_solution[i:i+3]) for i in range(0, 9, 3)] # Calculate row sums
                col_sums = [sum(best_solution[i::3]) for i in range(3)] # Calculate column sums
                diagonal_sums = [sum(best_solution[::4]), sum(best_solution[2:8:2])]  # Calculate diagonal sums
                if all(s == 15 for s in row_sums) and all(s == 15 for s in col_sums) and all(s == 15 for s in diagonal_sums): # Check if all sums are equal to the target sum
                    print("Best Solution:")
                    for i in range(0, 9, 3):
                        print(best_solution[i:i+3])
                    return
            generation += 1
        print("No possible solution available.")

if __name__ == "__main__":

    # Parameters for the genetic algorithm
    population_size = 20
    mutation_rate = 1
    crossover_rate = 1
    max_generations = 1000

    # Initialize the genetic algorithm
    ga = MovingMagicSquareGA(population_size, mutation_rate, crossover_rate, max_generations)

    # Measure execution time and memory usage
    start_time = time.time()

    initial_population = ga.generate_initial_population()
    population_size_bytes = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / (1024 * 1024)

    ga.genetic_algorithm()

    end_time = time.time()

    execution_time = end_time - start_time


    print("Time complexity: ", execution_time)
    print("Memory usage of the population data structure: {:.2f} MB".format(population_size_bytes))